Pip install opencv

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


## Importing Needed Libraries

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from numba import jit, cuda

## First storing data for preprocesssing

In [3]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [4]:
from keras.preprocessing import image

for j in range(43,45):
    path1 = os.path.join(cur_path,'train',str(j))
    directory = os.listdir(path1)
    for i in directory:
        try:
            test_image = image.load_img(f"{path1}\{i}", target_size = (30, 30))
            test_image = image.img_to_array(test_image)
            data.append(list(test_image))
            labels.append(j)
        except:
            print("error")


## Further Preprocessing 

In [5]:

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

# # print(data)
print(data.shape, labels.shape)
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=44)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# #Converting the labels into one hot encoding
y_train = to_categorical(y_train, 45)
y_test = to_categorical(y_test, 45)

(47209, 30, 30, 3) (47209,)


# Building the model

In [6]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(45, activation='softmax'))


##Compilation of Model

In [9]:
# #Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# @jit(target ="cuda")
# epoch = 50
#more accuracy increase the epochs size
history = model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
591/591 [==============================] - 52s 88ms/step - loss: 0.2928 - accuracy: 0.8791 - val_loss: 0.1637 - val_accuracy: 0.9045
Epoch 2/20
591/591 [==============================] - 55s 94ms/step - loss: 0.2940 - accuracy: 0.8783 - val_loss: 0.1622 - val_accuracy: 0.9081
Epoch 3/20
591/591 [==============================] - 55s 94ms/step - loss: 0.2874 - accuracy: 0.8808 - val_loss: 0.1650 - val_accuracy: 0.9147
Epoch 4/20
591/591 [==============================] - 55s 93ms/step - loss: 0.2640 - accuracy: 0.8872 - val_loss: 0.1559 - val_accuracy: 0.9180
Epoch 5/20
591/591 [==============================] - 53s 90ms/step - loss: 0.2609 - accuracy: 0.8875 - val_loss: 0.1632 - val_accuracy: 0.9135
Epoch 6/20
591/591 [==============================] - 55s 94ms/step - loss: 0.2796 - accuracy: 0.8827 - val_loss: 0.1799 - val_accuracy: 0.9024
Epoch 7/20
591/591 [==============================] - 55s 94ms/step - loss: 0.2766 - accuracy: 0.8837 - val_loss: 0.1580 - val_accuracy:

## SAving the Model for further Use

In [ ]:
model.save("Alert_classifier.h5")